# FYSS5120 Efficient Numerical Programming - Demo 3

Author: Felix Cahyadi

Creation date: 26.09.2023

Last changes: 27.09.2023 23:54

### Problem 1: Study the program aitken_accelerate.py, which applies Aitken’s $\Delta^2$ acceleration to two series, which slowly converge to $\pi/4$ and $\pi^2/6$, respectively. Answer the questions:

In [1]:
# Define simplified function
import itertools
def test():
    for i in itertools.cycle([1,-1]):
        yield i

Part 1, why does the generator produce the wrong sequence in one case, but right in another?

To answer that, we create 3 test cases, the first and second are similar to the one in demo. In the third one, we assign the generator to variables multiple times

In [2]:
# This is the first case
print('1st test case')
print(next(test()))
print(test())
print(next(test()))
print(test())
print(next(test()))
print(test())
print(50*'-')

# This is the second case
print('2nd test case')
tt = test()
print(tt)
print(next(tt))
print(tt)
print(next(tt))
print(tt)
print(next(tt))
print(50*'-')

# Create an extra test case
print('3rd test case')
tt1 = test()
print(next(tt1))
print(tt1)
tt2 = test()
print(next(tt2))
print(tt2)
tt3 = test()
print(next(tt3))
print(tt3)
print(50*'-')


1st test case
1
<generator object test at 0x000001E01CDD4740>
1
<generator object test at 0x000001E01CDD4740>
1
<generator object test at 0x000001E01CDD4740>
--------------------------------------------------
2nd test case
<generator object test at 0x000001E01CDD4890>
1
<generator object test at 0x000001E01CDD4890>
-1
<generator object test at 0x000001E01CDD4890>
1
--------------------------------------------------
3rd test case
1
<generator object test at 0x000001E01CDD4970>
1
<generator object test at 0x000001E01CDD49E0>
1
<generator object test at 0x000001E01CDD4A50>
--------------------------------------------------


Conclusion: From the 3 test cases, only the second case produce the correct sequence. It is because we are able to keep the state of the generator, and request for the next state.

We are not able to do that in the first and the third test case. In the third case, it's obvious that the generator can't keep its state because we keep defining a new generator, as we can see from the different memory address. While in the first case, although it has the same memory address, the generator can't keep its state because we keep redefining it each time, so it keeps giving the 1st element of the sequence. 

Part 2: What does itertools.islice(leibnitz_pi(),N)) do ? How would
you write that as a plain for-loop?

We'll copy the functions here

In [3]:
import itertools

def aitken(seq):
    """ 
    apply Aitken's method to sequence seq with terms {s_i}
     s_i' = s_i - (s_i+1 - s_i)^2/(s_i+2 - 2 s_i+1 + s_i)
    """
    s_i   = next(seq)  
    s_ip1 = next(seq)  
    s_ip2 = next(seq) 
    while True:
        yield s_i - (s_ip1 - s_i)**2/(s_ip2 - 2*s_ip1 + s_i)
        s_i, s_ip1, s_ip2 = s_ip1, s_ip2, next(seq)


def leibnitz_pi():
    """
    Leibnitz formula
     pi/4 = sum_k=0^inf (-1)**k/(2k+1)
    the partial sum is 
     s_N = sum_k=0^N (-1)**k/(2k+1)
    """
    sN = 0
    j = 1
    for i in itertools.cycle([1,-1]):
        yield sN
        sN += i/j
        j  += 2

def pi2_over_six():
    """
     pi^2/6 = sum_k=1^inf 1/k^2
    the partial sum is 
     s_N = sum_k=1^N 1/k^2
    """
    sN = 1
    k = 1
    while True:
        yield sN
        k += 1
        sN += 1/k**2       



We know that the function leibnitz_pi() is a generator, and itertools.islice will return an iterator.

Hence, when we are calling itertools.islice(leibnitz_pi(),N), $N$ acts as the stop value. islice is extracting the first $N$ values from leibnitz_pi() and turn it into iterator. From there, we can use the list(iterator) to turn the iterator into a list

In [4]:
N = 10
    
leibnitz_iter = itertools.islice(leibnitz_pi(),N)
print(leibnitz_iter) #itertools.islice object

We can demonstrate that it is an iterator using the code below:

In [5]:
for leib in leibnitz_iter:
    print(leib)

0
1.0
0.6666666666666667
0.8666666666666667
0.7238095238095239
0.8349206349206351
0.7440115440115441
0.8209346209346211
0.7542679542679545
0.8130914836797192


Next, I'm going to write itertools.islice(leibnitz_pi(),N) as a plain for loop. Here, the idea is to use list comprehension, and then use iter() to turn it into iterables.

To limit the number of elements to $N$, we are going to use zip(range(N),leibnitz_pi())

After that, we are going to iterate it, and only taking the element for the list comprehension.

In [6]:
# Using list comprehension
manual_iter = iter([elem for i,elem in zip(range(N),leibnitz_pi())])
print(manual_iter)

In [7]:
# Test manual_iter
for maniter in manual_iter:
    print(maniter)

0
1.0
0.6666666666666667
0.8666666666666667
0.7238095238095239
0.8349206349206351
0.7440115440115441
0.8209346209346211
0.7542679542679545
0.8130914836797192


Or in a more traditional for loop

In [8]:
manual_list = []
leib_gen = leibnitz_pi() # Define the generator
for i in range(N):
    manual_list.append(next(leib_gen)) # Append the values

print(manual_list) # We can turn it into a generator if we want

[0, 1.0, 0.6666666666666667, 0.8666666666666667, 0.7238095238095239, 0.8349206349206351, 0.7440115440115441, 0.8209346209346211, 0.7542679542679545, 0.8130914836797192]


Part 3: How does the function aitken(seq) work?

Let's look into their output

In [9]:
N2 = 50

res_lei = list(itertools.islice(leibnitz_pi(),N2))
res_ait = list(itertools.islice(aitken(leibnitz_pi()),N2))
print(res_lei)
print(res_ait)

[0, 1.0, 0.6666666666666667, 0.8666666666666667, 0.7238095238095239, 0.8349206349206351, 0.7440115440115441, 0.8209346209346211, 0.7542679542679545, 0.8130914836797192, 0.7604599047323508, 0.8080789523513985, 0.7646006914818333, 0.8046006914818333, 0.7675636544447964, 0.802046413065486, 0.769788348549357, 0.8000913788523872, 0.7715199502809587, 0.7985469773079856, 0.77290595166696, 0.797296195569399, 0.7740403816159106, 0.7962626038381329, 0.774986008093452, 0.7953941713587581, 0.7757863282215032, 0.7946542527498051, 0.7764724345679869, 0.7940162942171096, 0.7770671416747368, 0.7934605842976876, 0.7775875684246718, 0.7929721838092871, 0.7780468106749587, 0.7925395642981471, 0.7784550572558936, 0.7921536873928798, 0.7788203540595465, 0.7918073670465595, 0.7791491391984583, 0.7914948182108039, 0.7794466254397195, 0.7912113313220724, 0.7797170784485092, 0.790953033504689, 0.7799640225156781, 0.7907167106877211, 0.7801903948982474, 0.7904996732487628]
[0.75, 0.7916666666666667, 0.783333333

As we can see, the Aitken's method accelerates the convergence of the series by replacing the terms $s_i$. I'm going to comment on each line of aitken(seq) below to explain how it works. It's creating another generator

In [10]:
def aitken(seq):
    """ 
    apply Aitken's method to sequence seq with terms {s_i}
     s_i' = s_i - (s_i+1 - s_i)^2/(s_i+2 - 2 s_i+1 + s_i)
    """
    s_i   = next(seq) # At first, get the first 3 elements from generator seq
    s_ip1 = next(seq) # Second element 
    s_ip2 = next(seq) # Third element
    while True:
        yield s_i - (s_ip1 - s_i)**2/(s_ip2 - 2*s_ip1 + s_i) # Yield the calculated value for the new sequence
        s_i, s_ip1, s_ip2 = s_ip1, s_ip2, next(seq) # Rearranging the value of the variables for the next calculation, and acquiring the next element from seq

### The code demo3_heat_animation.py animates heat flow under the assumption that the temperature of every element is the average of its own temperature and of elements next to it, while keeping the outer edges at fixed temperature. That makes 5 elements to average over. Answer the questions:

Part 1: The method step() does one step of heat flow, and it’s the most
important part of the code. Please explain how it works, even without
any for-loops?

Firstly, we know that the attribute self.heat_map is defined by an np.array, where each element denotes the temperature at each node. Because of that, we can extract the nodes that we needed, and then average them by summing the arrays.

In [11]:
# For annotation

'''
def step(self):        
        mid    = self.heat_map[1:-1,1:-1] # Acquire the nodes in the middle of the graph, starting from the second row to the second last row, and second column to the last second column

        above  = self.heat_map[:-2,1:-1] # Similar to mid, but shift one row up, so we start from the first row and ends at the third last row.

        below  = self.heat_map[2:,1:-1] # Similar to mid, but shift one row down

        right  = self.heat_map[1:-1,:-2] # Similar to mid, but shift one column to the right

        left   = self.heat_map[1:-1,2:] # Similar to mid, but shift one column to the left

        mid[:] = (mid+above+below+left+right)/5 # Sum the arrays together and take the average
        
        return mid
'''

'\ndef step(self):        \n        mid    = self.heat_map[1:-1,1:-1] # Acquire the nodes in the middle of the graph, starting from the second row to the second last row, and second column to the last second column\n\n        above  = self.heat_map[:-2,1:-1] # Similar to mid, but shift one row up, so we start from the first row and ends at the third last row.\n\n        below  = self.heat_map[2:,1:-1] # Similar to mid, but shift one row down\n\n        right  = self.heat_map[1:-1,:-2] # Similar to mid, but shift one column to the right\n\n        left   = self.heat_map[1:-1,2:] # Similar to mid, but shift one column to the left\n\n        mid[:] = (mid+above+below+left+right)/5 # Sum the arrays together and take the average\n        \n        return mid\n'

In my opinion, the right should be 'right = self.heat_map[1:-1, 2:]' and the left should be 'left   = self.heat_map[1:-1,:-2]' instead.

The averaging makes everything okay though, since it produces the same effect

Part 2: Why should it be mid[:] = (mid+above+below+left+right)/5? Why doesn't mid = (mid+above+below+left+right)/5 work as intended?

Let's test this with the the modified Heat2D class, I added some print function in it

In [12]:
import numpy as np

class Heat2D:
    def __init__(self, height, width):
        # index order in heat_map is [y,x] to make matshow show it correctly
        self.heat_map = np.zeros((height+2,width+2),dtype=np.float64)
        self.heat_map[:] = 100.0 # initial temperature
        # dip to very cold environment, top stays at room temperature
        self.heat_map[:,0]  = -196.00
        self.heat_map[:,-1] = -196.00
        self.heat_map[-1,:] = -196.00
        self.heat_map[0,:]  = 22.0  
    
    def step(self):
        mid    = self.heat_map[1:-1,1:-1]
        print("This is the id before averaging: ", id(mid)) # Added print function
        above  = self.heat_map[:-2,1:-1]
        below  = self.heat_map[2:,1:-1]
        right  = self.heat_map[1:-1,:-2]
        left   = self.heat_map[1:-1,2:]
        mid[:] = (mid+above+below+left+right)/5
        print("This is the id after averaging: ",id(mid)) # Added print function
        return mid
    
mat = Heat2D(3,3)

In [13]:
for i in range(5):
    mat.step()

This is the id before averaging:  2062441444912
This is the id after averaging:  2062441444912
This is the id before averaging:  2062441444912
This is the id after averaging:  2062441444912
This is the id before averaging:  2062441444912
This is the id after averaging:  2062441444912
This is the id before averaging:  2062441444912
This is the id after averaging:  2062441444912
This is the id before averaging:  2062441444912
This is the id after averaging:  2062441444912


As we can see, the id of mid is always the same, this is because by using mid[:], we assign the value to the same array in the computer's memory. Here's what happens:
* mid = self.heat_map[1:-1,1:-1] initialize the array with the value from self.heat_map
* mid[:] = (mid+above+below+left+right)/5 assigns the value of the average to the mid, which in turns updates the value of self.heat_map[1:-1,1:-1]

And it repeats again, it is similar to what happens in the example below

In [14]:
arr1 = np.array([[1,2,3],[4,5,6],[7,8,9]],dtype=np.float64) # Create an array
arr2 = arr1[:,0] # refer parts of the array as arr2
arr2[:] = np.array([100,100,100]) # assign new value
print(arr1) # The original array now changes in value

[[100.   2.   3.]
 [100.   5.   6.]
 [100.   8.   9.]]


Now, we will create the class Heat2D_test, where we changed mid[:] to mid

In [15]:
class Heat2D_test:
    def __init__(self, height, width):
        # index order in heat_map is [y,x] to make matshow show it correctly
        self.heat_map = np.zeros((height+2,width+2),dtype=np.float64)
        self.heat_map[:] = 100.0 # initial temperature
        # dip to very cold environment, top stays at room temperature
        self.heat_map[:,0]  = -196.00
        self.heat_map[:,-1] = -196.00
        self.heat_map[-1,:] = -196.00
        self.heat_map[0,:]  = 22.0  
    
    def step(self):        
        mid    = self.heat_map[1:-1,1:-1]
        print("This is the id before averaging: ", id(mid))
        above  = self.heat_map[:-2,1:-1]
        below  = self.heat_map[2:,1:-1]
        right  = self.heat_map[1:-1,:-2]
        left   = self.heat_map[1:-1,2:]
        mid = (mid+above+below+left+right)/5 # I changed mid[:] to mid
        print("This is the id after averaging: ", id(mid))

        return mid
    
mat_test = Heat2D_test(200,200)

In [16]:
for i in range(5):
    mat_test.step()

This is the id before averaging:  2062441445104
This is the id after averaging:  2062441447312
This is the id before averaging:  2062441447312
This is the id after averaging:  2062441445104
This is the id before averaging:  2062441445104
This is the id after averaging:  2062441447312
This is the id before averaging:  2062441447312
This is the id after averaging:  2062441445104
This is the id before averaging:  2062441445104
This is the id after averaging:  2062441447312


As we can see, the id keeps changing, it's because when we do mid = (mid+above+below+left+right)/5, we make the variable mid refers to the new array instead. Here's what happens:
* mid = self.heat_map[1:-1,1:-1] initialize the array with the value from self.heat_map
* mid = (mid+above+below+left+right)/5, mid now refers to another thing, self.heat_map remains unchanged

self.heat_map doesn't change now matter how many times we iterate